<a href="https://colab.research.google.com/github/prakrutpatel/Context-RCNN-Tortoises/blob/main/Embedding%20Context%20Maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [ ]:
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
!pip install python-snappy

In [ ]:
!pip install apache-beam

In [ ]:
!pip install pycocotools

#### Download and install TensorFlow framework

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
%cd models/research/

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

## Import packages

In [ ]:
import numpy as np
import os
import six
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
import json
import matplotlib.pyplot as plt
from datetime import datetime
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

##### Optional - Not needed if your images are in tfrecords format with annotations and timestamp field

In [ ]:
#@title Optional, if you're starting for Json COCO format files { display-mode: "both" }
!python object_detection/dataset_tools/context_rcnn/create_cococameratraps_tfexample_main.py \
  --alsologtostderr \
  --output_tfrecord_prefix="/content/tfrecords_valid/" \
  --image_directory="/content/models/research/train" \
  --input_annotations_file="/content/models/research/train/_annotations.coco.json"

In [ ]:
#@title Optional, if you don't have bounding box for your data { display-mode: "both" }
!python object_detection/dataset_tools/context_rcnn/generate_detection_data.py \
  --alsologtostderr \
  --detection_input_tfrecord="/content/tfrecords_valid/" \
  --detection_output_tfrecord="/content/tfrecords_new" \
  --detection_model_dir="/content/drive/MyDrive/4/saved_model"

#### Generate Embddings for training and testing tfrecords file using Faster RCNN **input_type tfexample** model

The input node must of type tf.Example proto.
Reference - [Generate Embedding](https://github.com/tensorflow/models/blob/master/research/object_detection/dataset_tools/context_rcnn/generate_embedding_data.py)

In [ ]:
#Input parameters
faster_rcnn_model_path = "/content/drive/MyDrive/Faster RCNN/Faster_RCNN_SS1_tfexample/saved_model"

In [ ]:
#Update path as required
!python object_detection/dataset_tools/context_rcnn/generate_embedding_data.py \
    --alsologtostderr \
    --embedding_input_tfrecord="/content/drive/MyDrive/B004_long_term_tfrecords/B004_context_rcnn_memory_train" \
    --embedding_output_tfrecord="/content/B004_context_rcnn_memory_train_embedding" \
    --embedding_model_dir=faster_rcnn_model_path

In [ ]:
!python object_detection/dataset_tools/context_rcnn/generate_embedding_data.py \
    --alsologtostderr \
    --embedding_input_tfrecord="/content/drive/MyDrive/B004_long_term_tfrecords/B004_context_rcnn_memory_test" \
    --embedding_output_tfrecord="/content/B004_context_rcnn_memory_test_embedding" \
    --embedding_model_dir=faster_rcnn_model_path

##### Optional - Copy the resulting files to a permanent storage location

In [ ]:
!cp "/content/B004_context_rcnn_memory_train_embedding-00000-of-00001" "/content/drive/MyDrive/B004_long_term_tfrecords/B004_context_rcnn_memory_train_embedding-00001-of-00001"

In [ ]:
!cp "/content/B004_context_rcnn_memory_test_embedding-00000-of-00001" "/content/drive/MyDrive/B004_long_term_tfrecords/B004_context_rcnn_memory_test_embedding-00001-of-00001"

#### Add context for training and testing tfrecords file with embeddings (The files made in the steps above)

This tool groups images containing bounding boxes and embedded context features
by a key, either `image/location` or `image/seq_id`, and time horizon,
then uses these groups to build up a contextual memory bank from the embedded
context features from each image in the group and adds that context to the
output tf.Examples for each image in the group.

See reference [Add Context](https://github.com/tensorflow/models/blob/master/research/object_detection/dataset_tools/context_rcnn/add_context_to_examples.py)

In [ ]:
!python object_detection/dataset_tools/context_rcnn/add_context_to_examples.py \
  --input_tfrecord="/content/B004_context_rcnn_memory_train_embedding-00000-of-00001" \
  --output_tfrecord="/content/B004_context_rcnn_memory_train_embedding_with_context" \
  --sequence_key image/seq_id \
  --time_horizon month \
  --output_type tf_example

In [ ]:
!python object_detection/dataset_tools/context_rcnn/add_context_to_examples.py \
  --input_tfrecord="/content/B004_context_rcnn_memory_test_embedding-00000-of-00001" \
  --output_tfrecord="/content/B004_context_rcnn_memory_test_embedding_with_context" \
  --sequence_key image/seq_id \
  --time_horizon month \
  --output_type tf_example

##### Copy the resulting files to a permanent storage location to be used later

In [ ]:
!cp "/content/B004_context_rcnn_memory_train_embedding_with_context-00000-of-00001" "/content/drive/MyDrive/B004_long_term_tfrecords/B004_context_rcnn_memory_train_embedding_with_context-00001-of-00001"

In [ ]:
!cp "/content/B004_context_rcnn_memory_test_embedding_with_context-00000-of-00001" "/content/drive/MyDrive/B004_long_term_tfrecords/B004_context_rcnn_memory_test_embedding_with_context-00001-of-00001"